# Getting Started
This tutorial demonstrates the configuration and use of a simple BSK-RL environment.
BSK-RL and dependencies should already be installed at this point (see [Installation](../install.rst)
if you haven't installed the package yet).

## Load Modules
In this tutorial, the environment will be created with `gym.make`, so it is necessary to
import the top-level `bsk_rl` module as well as `gym` and `bsk_rl` components.

In [1]:
import gymnasium as gym
import numpy as np
from bsk_rl import act, data, obs, scene, sats
from bsk_rl.sim import dyn, fsw

from Basilisk.architecture import bskLogging
bskLogging.setDefaultLogLevel(bskLogging.BSK_WARNING)


If no errors were raised, you have a functional installation of `bsk_rl`.

## Configure the Satellite
[Satellites](../api_reference/sats/index.rst) are configurable agents in the environment.
To make a new environment, start by specifying the [observations](../api_reference/obs/index.rst)
and [actions](../api_reference/act/index.rst) of a satellite type, as well as the underlying
Basilisk [simulation](../api_reference/sim/index.rst) models used by the satellite.

In [2]:
class MyScanningSatellite(sats.AccessSatellite):
    observation_spec = [
        obs.SatProperties(
            dict(prop="storage_level_fraction"),
            dict(prop="battery_charge_fraction")
        ),
        obs.Eclipse(),
    ]
    action_spec = [
        act.Scan(duration=60.0),  # Scan for 1 minute
        act.Charge(duration=600.0),  # Charge for 10 minutes
    ]
    dyn_type = dyn.ContinuousImagingDynModel
    fsw_type = fsw.ContinuousImagingFSWModel

Based on this class specification, a list of configurable parameters for the satellite
can be generated.

In [3]:
MyScanningSatellite.default_sat_args()

{'hs_min': 0.0,
 'maxCounterValue': 4,
 'thrMinFireTime': 0.02,
 'desatAttitude': 'sun',
 'controlAxes_B': [1, 0, 0, 0, 1, 0, 0, 0, 1],
 'thrForceSign': 1,
 'K': 7.0,
 'Ki': -1,
 'P': 35.0,
 'imageAttErrorRequirement': 0.01,
 'imageRateErrorRequirement': None,
 'inst_pHat_B': [0, 0, 1],
 'utc_init': 'this value will be set by the world model',
 'batteryStorageCapacity': 288000.0,
 'storedCharge_Init': <function bsk_rl.sim.dyn.BasicDynamicsModel.<lambda>()>,
 'disturbance_vector': None,
 'dragCoeff': 2.2,
 'imageTargetMaximumRange': -1,
 'instrumentBaudRate': 8000000.0,
 'instrumentPowerDraw': -30.0,
 'basePowerDraw': 0.0,
 'wheelSpeeds': <function bsk_rl.sim.dyn.BasicDynamicsModel.<lambda>()>,
 'maxWheelSpeed': inf,
 'u_max': 0.2,
 'rwBasePower': 0.4,
 'rwMechToElecEfficiency': 0.0,
 'rwElecToMechEfficiency': 0.5,
 'panelArea': 1.0,
 'panelEfficiency': 0.2,
 'nHat_B': array([ 0,  0, -1]),
 'mass': 330,
 'width': 1.38,
 'depth': 1.04,
 'height': 1.58,
 'sigma_init': <function bsk_rl.sim

When instantiating a satellite, these parameters can be overriden with a constant or 
rerandomized every time the environment is reset using the ``sat_args`` dictionary.

In [4]:
sat_args = {}

# Set some parameters as constants
sat_args["imageAttErrorRequirement"] = 0.05
sat_args["dataStorageCapacity"] = 1e10
sat_args["instrumentBaudRate"] = 1e7
sat_args["storedCharge_Init"] = 50000.0

# Randomize the initial storage level on every reset
sat_args["storageInit"] = lambda: np.random.uniform(0.25, 0.75) * 1e10

# Make the satellite
sat = MyScanningSatellite(name="EO1", sat_args=sat_args)

## Making the Environment
For this example, we will be using the single-agent [SatelliteTasking](../api_reference/index.rst) 
environment. Along with passing the satellite that we configured, the environment takes
a [scenario](../api_reference/scene/index.rst), which defines the environment the
satellite is acting in, and a [rewarder](../api_reference/data/index.rst), which defines
how data collected from the scenario is rewarded.

In [5]:
env = gym.make(
    "SatelliteTasking-v1",
    satellite=sat,
    scenario=scene.UniformNadirScanning(),
    rewarder=data.ScanningTimeReward(),
    time_limit=5700.0,  # approximately 1 orbit
    log_level="INFO",
)

2024-09-11 09:27:27,708 gym                            INFO       Calling env.reset() to get observation space


2024-09-11 09:27:27,708 gym                            INFO       Resetting environment with seed=2880513408


2024-09-11 09:27:27,788 sats.satellite.EO1             INFO       <0.00> EO1: Finding opportunity windows from 0.00 to 5700.00 seconds


2024-09-11 09:27:27,803 gym                            INFO       <0.00> Environment reset


## Interacting with the Environment

First, the environment is reset.

In [6]:
observation, info = env.reset(seed=1)

2024-09-11 09:27:27,908 gym                            INFO       Resetting environment with seed=1


2024-09-11 09:27:28,077 sats.satellite.EO1             INFO       <0.00> EO1: Finding opportunity windows from 0.00 to 5700.00 seconds


2024-09-11 09:27:28,089 gym                            INFO       <0.00> Environment reset


Next, we take the scan action (`action=0`) a few times. This allows for the satellite to
settle its attitude in the nadir pointing mode to satisfy imaging conditions. Note that 
the logs show little or no data accumulated in the first two steps as it settles, but
achieves 60 reward (corresponding to 60 seconds of imaging) by the third step.

In [7]:
print("Initial data level:", observation[0], "(randomized by sat_args)")
for _ in range(3):
    observation, reward, terminated, truncated, info = env.step(action=0)
print("  Final data level:", observation[0])

2024-09-11 09:27:28,093 gym                            INFO       <0.00> === STARTING STEP ===


2024-09-11 09:27:28,093 sats.satellite.EO1             INFO       <0.00> EO1: action_nadir_scan tasked for 60.0 seconds


2024-09-11 09:27:28,093 sats.satellite.EO1             INFO       <0.00> EO1: setting timed terminal event at 60.0


2024-09-11 09:27:28,097 sats.satellite.EO1             INFO       <60.00> EO1: timed termination at 60.0 for action_nadir_scan


2024-09-11 09:27:28,097 data.base                      INFO       <60.00> Data reward: {'EO1': 0.0}


...

2024-09-11 09:27:28,108 sats.satellite.EO1             INFO       <180.00> EO1: timed termination at 180.0 for action_nadir_scan


2024-09-11 09:27:28,108 data.base                      INFO       <180.00> Data reward: {'EO1': 60.0}


2024-09-11 09:27:28,108 sats.satellite.EO1             INFO       <180.00> EO1: Satellite EO1 requires retasking


2024-09-11 09:27:28,109 gym                            INFO       <180.00> Step reward: 60.0


Initial data level: 0.7341307878 (randomized by sat_args)
  Final data level: 0.8241307878


The observation reflects the increase in stored data. The first element, corresponding
to `storage_level_fraction`, starts at a random value set by the `storageInit` function
in `sat_args` and increases based on the time spent imaging.

Finally, the charging mode is tasked repeatedly in 10-minute increments until the
environment time limit is reached.

In [8]:
while not truncated:
    observation, reward, terminated, truncated, info = env.step(action=1)
    print(f"Charge level: {observation[1]:.3f} ({env.unwrapped.simulator.sim_time:.1f} seconds)\n\tEclipse: start: {observation[2]:.1f} end: {observation[3]:.1f}")

2024-09-11 09:27:28,112 gym                            INFO       <180.00> === STARTING STEP ===


2024-09-11 09:27:28,112 sats.satellite.EO1             INFO       <180.00> EO1: action_charge tasked for 600.0 seconds


2024-09-11 09:27:28,113 sats.satellite.EO1             INFO       <180.00> EO1: setting timed terminal event at 780.0


2024-09-11 09:27:28,144 sats.satellite.EO1             INFO       <780.00> EO1: timed termination at 780.0 for action_charge


2024-09-11 09:27:28,144 data.base                      INFO       <780.00> Data reward: {'EO1': 0.0}


...

2024-09-11 09:27:28,469 data.base                      INFO       <5700.00> Data reward: {'EO1': 0.0}


2024-09-11 09:27:28,469 gym                            INFO       <5700.00> Step reward: 0.0


2024-09-11 09:27:28,470 gym                            INFO       <5700.00> Episode terminated: False


2024-09-11 09:27:28,470 gym                            INFO       <5700.00> Episode truncated: True


Charge level: 1.000 (4380.0 seconds)
	Eclipse: start: 1740.0 end: 3870.0
Charge level: 1.000 (4980.0 seconds)
	Eclipse: start: 1140.0 end: 3270.0
Charge level: 1.000 (5580.0 seconds)
	Eclipse: start: 540.0 end: 2670.0
Charge level: 1.000 (5700.0 seconds)
	Eclipse: start: 420.0 end: 2550.0


It is observed that the battery decrease while the satellite is in eclipse, but once the
satellite is out of eclipse, the battery quickly increases to full charge.